# Uso práctico de Spark


In [0]:
from pyspark.sql import functions as f

## 1 - Carga de datos

Lea los datos dispoibles en la siguiente ubicación:

In [0]:
df = spark.read.format("json").load("wasbs://sid@yjtgeneral.blob.core.windows.net/cafeteria")

df.printSchema()

Si conocemos el esquema podemos fijarlo nosotros mismos

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, LongType, BooleanType, ArrayType

schema = StructType([
    StructField("paid", StructType([
        StructField("createdOn", StringType(), True),
        StructField("paymentMethodValue", ArrayType(
            StructType([
                StructField("_id", StringType(), True),
                StructField("approbationNumber", StringType(), True),
                StructField("cashBox", StringType(), True),
                StructField("createdOn", StringType(), True),
                StructField("internalId", StringType(), True),
                StructField("paymentMethod", StringType(), True),
                StructField("tip", LongType(), True),
                StructField("value", LongType(), True)
            ])
        ), True)
    ]), True),
    StructField("products", ArrayType(
        StructType([
            StructField("_id", StringType(), True),
            StructField("avgCost", LongType(), True),
            StructField("categoryId", StringType(), True),
            StructField("categoryName", StringType(), True),
            StructField("name", StringType(), True),
            StructField("price", LongType(), True),
            StructField("quantity", LongType(), True),
            StructField("total", LongType(), True)
        ])
    ), True),
    StructField("table", StructType([
        StructField("idInternal", StringType(), True),
        StructField("name", StringType(), True)
    ]), True)
])

df = spark.read.format("json") \
    .schema(schema) \
    .load("wasbs://sid@yjtgeneral.blob.core.windows.net/cafeteria")

El esquema puede inferirse de una muestra de los datos con  `samplingRatio`

In [0]:
df = spark.read.format("json").option("samplingRatio", 0.1).load("wasbs://sid@yjtgeneral.blob.core.windows.net/cafeteria")

## 2 - Exploración

Imprimir el esquema

In [0]:
df.printSchema()

Contar el número total de registros

In [0]:
df.count()

Visualizar las primeras filas

In [0]:
display(df)

## Filtros, agregaciones y casts

Crear un DataFrame de ventas a nivel de producto utilizando `explode()`

In [0]:

ventas = (
    df.select(
        f.to_date("paid.createdOn").alias("fecha"),
        f.explode("products").alias("producto")
    )
    .withColumn("producto_id", f.col("producto._id"))
    .withColumn("producto_name", f.col("producto.name"))
    .withColumn("producto_quantity", f.col("producto.quantity"))
    .withColumn("producto_price", f.col("producto.price"))
    .drop("producto")
)

Calcular cuál fue el valor vendido de cada producto

In [0]:
display(
    ventas
    .groupby("producto_name")
    .agg(
        f.sum(f.col("producto_quantity") * f.col("producto_price")).alias("total")
    )
    .orderBy(f.desc("total"))
)

Almacenar una tabla en que cada fila represente un producto sin repetir 

¿Cómo debería particionarse esta tabla?

In [0]:
(
    ventas
    .select("producto_id", "producto_name")
    .distinct()
    .write
    .mode("overwrite")
    # En hive podríamos utilizar bucketing de la siguiente forma, si tuvieramos un gran número de productos:
    # .bucketBy(10, "producto_id")  
    .saveAsTable("productos")
)

Almacenar una tabla con las ventas que incluya un id del producto, la fecha y la cantidad.

¿Cómo debería particionarse esta tabla?

In [0]:
(
    ventas
    .select("producto_id", "fecha", "producto_quantity", "producto_price")
    .write.mode("overwrite")
    # En hive podríamos utilizar bucketing de la siguiente forma, si tuvieramos un gran número de productos:
    # .bucketBy(10, "producto_id")  
    .option("mergeSchema", "true") # Permite sobre escribir el esquema
    .saveAsTable("ventas")
)

Crear un dataframe con las ventas de julio.

In [0]:
df_ventas = spark.read.table("ventas")
df_productos = spark.read.table("productos")

In [0]:
ventas_julio = (
    df_ventas
    .filter(f.month("fecha") == 1)
)

display(ventas_julio)

## 3 - Joins

Leer las tablas de productos y ventas. Juntarlas y calcular el valor de las ventas por mes.

In [0]:
display(
    df_ventas
    .join(
        df_productos,
        on="producto_id",
        how="inner"
    )
    .groupby(f.month("fecha"))
    .agg(f.sum("producto_quantity"))
)

Identificar todos los productos que se vendieron cada mes pero no se vendieron en enero

In [0]:
display(
  ventas
  .select("producto_id", "producto_name", f.month("fecha"))
  .distinct()
  .join(
    ventas_julio,
    on="producto_id",
    how="anti"  
  )
)

## 4 - Ventanas

Mostrar el top 3 de los productos más vendidos cada mes

In [0]:
from pyspark.sql import Window

ventana = Window.partitionBy("mes").orderBy(f.desc("cantidad"))

display(
  ventas
  .groupby(f.month("fecha").alias("mes"), "producto_name")
  .agg(f.sum("producto_quantity").alias("cantidad"))
  .withColumn("rank", f.rank().over(ventana))
  .filter("rank <= 3")
  .orderBy("mes", "rank")
)

## 5 - Aproximaciones


Cuántos clientes han visitado la cafetería cada mes

In [0]:
display(
    df
    .groupby(f.month("createdOn"))
    .agg(f.approx_count_distinct("client_id", rsd=0.01))
)

Cuánto es lo minimos valor del 20% de las facturas más grandes

In [0]:
display(
    df
    .agg(
        # Calculamos una aproximación al percentil 80
        f.approx_percentile("totalPaid", 0.80)
    )
)

# Explore el dataset para encontrar `insights`